Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [4]:
from random import choices
import random
import lab9_lib

PROBLEM_SIZES = [1, 2, 5, 10]
LOCI = 1000

In [5]:
import copy
import numpy as np

class LocalSearch:
    def __init__(self, mut_rate, cmb_rate, blck_size, prb_size):
        self.mutation_rate = mut_rate
        self.combination_rate = cmb_rate
        #Block size expressed in percentage
        self.block_size = blck_size
        #Size retrieved in survival selection phase
        self.problem_size = prb_size
        self.population = []
        self.survivors = []

    #Here n_individuals could be grater than problme_size since population will be filtered in survival_selection()
    def generate_population(self, n_individuals):
        for _ in range(n_individuals):
            self.population.append(choices([0, 1], k=LOCI))

    #Function that receives a genome ad return a mutated version of it accordingly to a random choice and a mutation paremeter
    def mutate(self, genome):
        mutated_genome = []

        for e in genome:
            if random.random() > self.mutation_rate:
                mutated_genome.append(not e)
            else:
                mutated_genome.append(e)

        return mutated_genome

    #TO DO unisci queste due funzioni di combine

    #Function that receives two genomes and returns a combined version of the two.
    #The combination happens at a very specific level. In fact each element (bit from 
    # our poit of view) of the genome is taken into consideration before swapping.
    def combine_genomes_bit_level(self, gen_1, gen_2):
        combined_1 = []
        combined_2 = []

        for i in range(len(gen_1)):
            if random.random() > self.combination_rate:
                tmp = gen_1[i]
                gen_1[i] = gen_2[i]
                gen_2[i] = tmp

        return combined_1, combined_2

    #Function that receives two genomes and returns a combined version of the two.
    #The combination happens at block level. Same as above, but instead of varying
    #at bit level, it varies a sequence of bits.
    def combine_genomes_block_level(self, gen_1, gen_2):
        combined_1 = []
        combined_2 = []
        bound = int(self.block_size*len(gen_1))
        start = 0

        for _ in range(len(gen_1)/bound):
            if random.random() > self.combination_rate:
                tmp = gen_1[start:bound]
                gen_1[start:bound] = gen_2[start:bound]
                gen_2[start:bound] = tmp
            start += bound

        return combined_1, combined_2
    
    def combine_genomes_block_level_hermaphrodite(self, gen_1, gen_2):
        combined_1 = []
        combined_2 = []
        bound = int(self.block_size*len(gen_1))
        start = 0

        for _ in range(len(gen_1)/bound):
            if random.random() > self.combination_rate:
                #Sostituire il blocco in una posizione random
                
                

                print("ouec")

        return combined_1, combined_2
    
    #A seconda di herma.. dipende anche la combination (non ha senso mettere i blocchi nella stessa posizione, fai)
    def parent_selection(self, hermaphrodite=False):
        if hermaphrodite:
            #Hermaphrodite approach
            p_1 = self.population[random.random(len(self.population))]
            p_2 = self.population[random.random(len(self.population))]
        else:
            ppl = copy.deepcopy(self.population)
            p_1 = ppl[random.random(len(self.population))]
            ppl.remove(p_1)
            p_2 = ppl[random.random(len(self.population))]
        return p_1, p_2

    #Find the best prblem_size individuals among the population and mark them as survivors
    #Se fai così però generando nuovi individui devi reinserirli nella popolazione --> Vedi tu se sostituirli ai parent o meno
    def survival_selection(self):
        fits = []
        for ind in self.population:
            fits.append(lab9_lib.make_problem(ind))
        idxs = np.argmax(fits)
        self.survivors = self.population[idxs[:self.problem_size]]
        return self.survivors

In [6]:
for problem_size in PROBLEM_SIZES:
    fitness = lab9_lib.make_problem(problem_size)

    #In teoria al posto di questo for devi continuare finchè non risolvi, quindi diventa while
    for n in range(problem_size):
        ind = choices([0, 1], k=50)
        
        print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

    print(fitness.calls)

00000101011011110000001001110001111000111110010001: 46.00%
1
10100001101010101110110101000111110110100000100100: 26.00%
11010011001111000111001001001010001011111101001011: 25.40%
2
10011100010101110000100010111010111001111110010010: 12.67%
01110100001101001010001101001000101101011111010010: 12.69%
01000001110001101000000111011000011100011101111010: 12.71%
11011001000110111011011111100010001111110001101010: 26.69%
00010110001100100010100110000110110111001001111011: 12.69%
5
11111101101111101101110011000111100011011101000011: 19.13%
00001101110110110110110011000111111010110001110110: 9.11%
00101111100111010010111010101101111110111101111111: 9.11%
00011001000000111111010110101000100110101011100010: 15.34%
11000100110110101100001010100111010110000010000011: 23.56%
10101011010011101001110000110000111111111100110101: 31.34%
01001110011110000110011100010110101100110011100000: 7.33%
00010100000010111110000110101100010001101100011101: 29.56%
11001011011000001110110011010010001101110001001111: 9